## [Lambda](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Lambda) Layer walkthrough 

<table class="colab-notebook-buttons" align="center" width="50%";>
  <td><center>
    <a target="_blank" href="https://colab.research.google.com/github/Atharva-Gundawar/Huber_and_friends/blob/main/Huber_and_friends.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    </br>Run in Google Colab</a>
  </center></td>
  <td><center>
    <a target="_blank" href="https://github.com/Atharva-Gundawar/Huber_and_friends/blob/main/Huber_and_friends.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    </br>View source on GitHub</a>
  </center></td>
</table>

## Preparing the colab 

In [3]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np


In [4]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
)

Epoch 1/10
469/469 [==============================] - 2s 4ms/step - loss: 0.3518 - sparse_categorical_accuracy: 0.9037 - val_loss: 0.1961 - val_sparse_categorical_accuracy: 0.9437
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1667 - sparse_categorical_accuracy: 0.9527 - val_loss: 0.1378 - val_sparse_categorical_accuracy: 0.9609
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1201 - sparse_categorical_accuracy: 0.9657 - val_loss: 0.1118 - val_sparse_categorical_accuracy: 0.9669
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0938 - sparse_categorical_accuracy: 0.9729 - val_loss: 0.1021 - val_sparse_categorical_accuracy: 0.9702
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0755 - sparse_categorical_accuracy: 0.9784 - val_loss: 0.0847 - val_sparse_categorical_accuracy: 0.9750
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0613 - sparse_categorica

In [11]:
def abs_not_relu(x):
    return tf.keras.backend.abs(x)

In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Lambda(abs_not_relu),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
)

Epoch 1/10
469/469 [==============================] - 2s 4ms/step - loss: 0.3183 - sparse_categorical_accuracy: 0.9138 - val_loss: 0.1481 - val_sparse_categorical_accuracy: 0.9573
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1204 - sparse_categorical_accuracy: 0.9662 - val_loss: 0.1067 - val_sparse_categorical_accuracy: 0.9682
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0818 - sparse_categorical_accuracy: 0.9766 - val_loss: 0.0888 - val_sparse_categorical_accuracy: 0.9725
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0613 - sparse_categorical_accuracy: 0.9822 - val_loss: 0.0815 - val_sparse_categorical_accuracy: 0.9753
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0473 - sparse_categorical_accuracy: 0.9865 - val_loss: 0.0743 - val_sparse_categorical_accuracy: 0.9760
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0391 - sparse_categorica

You can notice a slight increase in the sparse categorical accuracy : 0.9916 -> 0.9963 